## This notebook provides a few cells needed for me to quickly update VF_WISESIZE_v#.fits, where # is some integer value representing the table version, using the most recent MAGPHYS and PHOT catalogs. 

In [1]:
import numpy as np
from astropy.table import Table
import os
homedir = os.getenv("HOME")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
magphys=Table.read(homedir+'/Desktop/v2-20220820/vf_v2_magphys_legacyExt_final.fits')
phot = Table.read(homedir+'/Desktop/galfit_files/vf_v2_legacy_ephot.fits')
alt_magphys=Table.read(homedir+'/Desktop/galfit_files/vf-altphot.fits')

In [3]:
vfw = Table.read(homedir+'/Desktop/galfit_files/VF_WISESIZE_v2.fits')

### updating SFR, Mstar, sSFR, and subsample flags

In [23]:
sfr_flag = np.zeros(len(vfw),dtype=bool)
mstar_flag = np.zeros(len(vfw),dtype=bool)
ssfr_flag = np.zeros(len(vfw),dtype=bool)

for i in range(len(vfw)):
    sfr_flag[i] = True if (alt_magphys['combined_logSFR_med'][i]>-1.014) else False
    mstar_flag[i] = True if (alt_magphys['combined_logMstar_med'][i]>8.26) else False
    ssfr_flag[i] = True if ((alt_magphys['combined_logSFR_med'][i] - alt_magphys['combined_logMstar_med'][i])>-11.5) else False

In [5]:
vfw['SFRflag'] = sfr_flag
vfw['massflag'] = mstar_flag
vfw['sSFR_flag'] = ssfr_flag

In [43]:
subsampleflag = (vfw['massflag']) & (vfw['SNRflag']) & (vfw['t_flag']) & (vfw['SFRflag']) & (vfw['sSFR_flag'])

In [21]:
vfw['subsample_flag'] = subsampleflag

In [22]:
vfw.write(homedir+'/Desktop/galfit_files/VF_WISESIZE_v2.fits',overwrite=True)

### updating group flags using PHOT catalog from Dr. Moustakas

In [27]:
groupnames = np.zeros(len(vfw),dtype='|S33')
ncomps=np.zeros(len(vfw),dtype=int)
primary_flags = np.zeros(len(vfw),dtype=bool)
group_flags = np.zeros(len(vfw),dtype=bool)
#groupIDs = np.zeros(len(vfw),dtype=int)

In [ ]:
for i in range(len(vfw)):
    if vfw['VFID'][i] in phot['VFID']:
        ncomp = phot[(phot['VFID']==vfw['VFID'][i])]['GROUP_MULT'].data[0]
        groupname = phot[(phot['VFID']==vfw['VFID'][i])]['GROUP_NAME'].data[0]
        primary_flag = phot[(phot['VFID']==vfw['VFID'][i])]['GROUP_PRIMARY'].data[0]
        
    else:
        groupname = 'N/A'
        ncomp=1
        group_flag=False
        primary_flag=False
        
    ncomps[i] = ncomp
    primary_flags[i] = primary_flag
    groupnames[i] = groupname
    print(i)

group_flag = ncomps>1

In [31]:
vfw['primaryGroup_flag'] = primary_flags
vfw['group_name'] = groupnames
vfw['ncomp'] = ncomps
vfw['group_flag'] = group_flags

### updating SNR flags using PHOT catalog from John Moustakas <br>
##### note: subsample_flag will now incorporate AP03 SNR rather than the photutils aperture SNR.

In [36]:
#phot = phot_all[phot_all['photFlag']]
    
w3_sig = phot['FLUX_AP03_W3']
w3_noise = 1/np.sqrt(phot['FLUX_IVAR_AP03_W3'])

snrphot = w3_sig/w3_noise
snrphot_flag = (snrphot>10)

/var/folders/gx/q5wqxlgs0rgfm6f87vgxlgyw4s52w0/T/ipykernel_2611/1745565839.py:4: RuntimeWarning: divide by zero encountered in divide
  w3_noise = 1/np.sqrt(phot['FLUX_IVAR_AP03_W3'])


In [40]:
vfw['SNRflag'] = snrphot_flag
try:
    vfw.add_column(snrphot,name='SNR_phot',index=11)
except:
    vfw['SNR_phot'] = snrphot

In [44]:
subsampleflag = (vfw['massflag']) & (vfw['SNRflag']) & (vfw['t_flag']) & (vfw['SFRflag']) & (vfw['sSFR_flag'])

In [45]:
vfw['subsample_flag'] = subsampleflag

In [46]:
vfw.write(homedir+'/Desktop/galfit_files/VF_WISESIZE_photSNR.fits',overwrite=True)